In [ ]:
import zipfile
from pathlib import Path

# Create submission zip
predictions_file = '/content/output/predictions.txt'
submission_zip = '/content/output/submission.zip'

print("📦 Creating submission.zip...")

with zipfile.ZipFile(submission_zip, 'w') as zipf:
    zipf.write(predictions_file, arcname='predictions.txt')

print(f"✅ Submission file created: {submission_zip}")
print(f"   File size: {os.path.getsize(submission_zip) / 1024:.2f} KB")

# Verify zip contents
print("\n📋 ZIP file contents:")
with zipfile.ZipFile(submission_zip, 'r') as zipf:
    for file_info in zipf.filelist:
        print(f"   {file_info.filename} ({file_info.file_size} bytes)")

# Copy to Google Drive
print("\n💾 Copying to Google Drive...")
import shutil
drive_submission = '/content/drive/MyDrive/LPR_Project/submission.zip'
shutil.copy(submission_zip, drive_submission)
print(f"✅ Submission saved to Google Drive: {drive_submission}")

# Also copy predictions.txt for reference
drive_predictions = '/content/drive/MyDrive/LPR_Project/predictions.txt'
shutil.copy(predictions_file, drive_predictions)
print(f"✅ Predictions saved to Google Drive: {drive_predictions}")

print("\n" + "="*60)
print("🎉 SUBMISSION READY!")
print("="*60)
print(f"✅ Submission file: {submission_zip}")
print(f"✅ Ready to upload to CodaBench")
print("\nInstructions:")
print("1. Download submission.zip from Google Drive")
print("2. Go to CodaBench competition page")
print("3. Navigate to 'My Submissions' tab")
print("4. Upload submission.zip")
print("5. Monitor leaderboard for results")

## 8. Create Submission ZIP File

In [ ]:
def generate_submission(model, inference_dataset, device, output_path='/content/output/predictions.txt'):
    """Generate submission predictions for all tracks"""
    
    predictions_list = []
    model.eval()
    
    print(f"🚀 Generating predictions for {len(inference_dataset)} tracks...")
    
    with torch.no_grad():
        for idx in tqdm(range(len(inference_dataset))):
            track_id, images = inference_dataset[idx]
            
            if not images:
                continue
            
            # Get predictions from all images
            predictions, confidences = predict_track(model, images, device)
            
            # Aggregate predictions
            final_pred, final_conf = aggregate_predictions(predictions, confidences)
            
            if final_pred.strip():  # Only save non-empty predictions
                predictions_list.append({
                    'track_id': track_id,
                    'plate_text': final_pred,
                    'confidence': final_conf
                })
    
    # Save predictions to file
    with open(output_path, 'w') as f:
        for pred in predictions_list:
            line = f"{pred['track_id']},{pred['plate_text']};{pred['confidence']:.4f}"
            f.write(line + '\n')
    
    print(f"✅ Predictions saved to {output_path}")
    print(f"   Total predictions: {len(predictions_list)}")
    
    return predictions_list


# Generate submission
inference_dataset = InferenceDataset(DATA_PATH, num_images=5)
predictions = generate_submission(model, inference_dataset, DEVICE)

# Display first 10 predictions
print("\nFirst 10 predictions:")
for pred in predictions[:10]:
    print(f"  {pred['track_id']},{pred['plate_text']};{pred['confidence']:.4f}")

## 7. Generate Submission File

In [ ]:
# Load best model for inference
model.load_state_dict(best_checkpoint)
model.eval()

print("✅ Best model loaded for inference")

class InferenceDataset(Dataset):
    """Dataset for inference on test tracks"""
    
    def __init__(self, data_root, num_images=5):
        self.samples = []
        self.num_images = num_images
        self._load_samples(data_root)
    
    def _load_samples(self, data_root):
        """Load all track paths"""
        for scenario in os.listdir(data_root):
            scenario_path = os.path.join(data_root, scenario)
            if not os.path.isdir(scenario_path):
                continue
            
            for layout in os.listdir(scenario_path):
                layout_path = os.path.join(scenario_path, layout)
                if not os.path.isdir(layout_path):
                    continue
                
                for track in os.listdir(layout_path):
                    track_path = os.path.join(layout_path, track)
                    if not os.path.isdir(track_path):
                        continue
                    
                    ann_path = os.path.join(track_path, 'annotations.json')
                    if os.path.exists(ann_path):
                        self.samples.append({
                            'track_path': track_path,
                            'track_id': track,
                        })
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        track_path = sample['track_path']
        track_id = sample['track_id']
        
        images = []
        for i in range(1, self.num_images + 1):
            img_path = os.path.join(track_path, f'lr-00{i}.png')
            if os.path.exists(img_path):
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (128, 32))
                image = image.astype(np.float32) / 255.0
                image = torch.from_numpy(image).unsqueeze(0)
                images.append(image)
        
        return track_id, images


def predict_track(model, images, device):
    """Predict on multiple images from a track"""
    predictions = []
    confidences = []
    
    with torch.no_grad():
        for image in images:
            image = image.to(device)
            logits = model(image.unsqueeze(0))  # Add batch dimension
            
            # Get prediction
            output = torch.argmax(logits, dim=2)  # (1, 32)
            
            # Greedy decoding
            prev, seq = -1, []
            for t in range(output.size(1)):
                val = output[0, t].item()
                if val != prev and val != 0:
                    seq.append(val)
                prev = val
            
            pred_text = decode_to_string(seq)
            
            # Calculate confidence as average softmax probability
            softmax = torch.softmax(logits, dim=2)  # (NUM_CLASSES, 1, 32)
            max_probs = torch.max(softmax, dim=0)[0]  # (1, 32)
            confidence = max_probs.mean().item()
            
            predictions.append(pred_text)
            confidences.append(confidence)
    
    return predictions, confidences


def aggregate_predictions(predictions, confidences):
    """Aggregate predictions from multiple images using confidence-based voting"""
    if not predictions:
        return "", 0.0
    
    # Filter out empty predictions
    valid_preds = [(p, c) for p, c in zip(predictions, confidences) if p.strip()]
    
    if not valid_preds:
        return "", 0.0
    
    # Majority voting with confidence weighting
    from collections import Counter
    pred_texts = [p for p, _ in valid_preds]
    
    # Count occurrences
    counter = Counter(pred_texts)
    most_common = counter.most_common(1)[0][0]
    
    # Get average confidence for most common
    avg_confidence = np.mean([
        c for p, c in valid_preds if p == most_common
    ])
    
    return most_common, avg_confidence

# Test inference on a single sample
print("\n🔍 Testing inference on a single sample...")
inference_dataset = InferenceDataset(DATA_PATH, num_images=5)
track_id, images = inference_dataset[0]
print(f"Track ID: {track_id}")
print(f"Number of images: {len(images)}")

predictions, confidences = predict_track(model, images, DEVICE)
print(f"Predictions: {predictions}")
print(f"Confidences: {[f'{c:.4f}' for c in confidences]}")

final_pred, final_conf = aggregate_predictions(predictions, confidences)
print(f"Final prediction: {final_pred}")
print(f"Final confidence: {final_conf:.4f}")

## 6. Inference & Prediction Aggregation

In [ ]:
# Training configuration
epochs = 25
lr = 1e-3
weight_decay = 1e-5

# Loss and optimizer
criterion = torch.nn.CTCLoss(blank=0, zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3, verbose=True
)

# Training history
history = {'train_loss': [], 'val_acc': []}
best_val_acc = 0.0
best_checkpoint = None

print("🚀 Starting training...")
print(f"   Epochs: {epochs}")
print(f"   Learning rate: {lr}")
print(f"   Device: {DEVICE}")

for epoch in range(epochs):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"{'='*60}")
    
    # Train
    train_loss = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
    history['train_loss'].append(train_loss)
    
    # Validate
    val_acc = validate(model, val_loader, DEVICE)
    history['val_acc'].append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Accuracy: {val_acc*100:.2f}%")
    
    # Learning rate scheduling
    scheduler.step(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_checkpoint = model.state_dict()
        checkpoint_path = f'/content/checkpoints/model_best.pt'
        torch.save(model.state_dict(), checkpoint_path)
        print(f"✅ Best model saved (accuracy: {val_acc*100:.2f}%)")

print(f"\n✅ Training completed!")
print(f"Best validation accuracy: {best_val_acc*100:.2f}%")

# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(history['val_acc'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.grid()

plt.tight_layout()
plt.savefig('/content/output/training_history.png', dpi=100)
plt.show()

In [ ]:
def collate_fn(batch):
    """Collate function for DataLoader"""
    images, labels, lengths, texts = zip(*batch)
    images = torch.stack(images)
    targets = torch.cat(labels)
    target_lengths = torch.tensor(lengths, dtype=torch.long)
    return images, targets, target_lengths, texts


def greedy_decoder(output):
    """Decode CTC output using greedy decoding"""
    # output: (T, B, C)
    output = torch.argmax(output, dim=2)  # (T, B)
    decoded = []
    
    for b in range(output.size(1)):
        prev, seq = -1, []
        for t in range(output.size(0)):
            val = output[t, b].item()
            if val != prev and val != 0:  # Skip blanks and consecutive duplicates
                seq.append(val)
            prev = val
        decoded.append(seq)
    
    return decoded


def decode_to_string(seq):
    """Convert indices to string"""
    return "".join(idx_to_char.get(i, '') for i in seq if i in idx_to_char)


def validate(model, val_loader, device):
    """Validate model on validation set"""
    model.eval()
    correct, total = 0, 0
    
    with torch.no_grad():
        for images, targets, target_lengths, texts in val_loader:
            images = images.to(device)
            logits = model(images)  # (NUM_CLASSES, B, 32)
            decoded = greedy_decoder(logits)
            
            # Calculate accuracy
            idx = 0
            for i, seq in enumerate(decoded):
                gt = decode_to_string(targets[idx:idx+target_lengths[i]].tolist())
                pred = decode_to_string(seq)
                
                if gt.strip() == pred.strip():
                    correct += 1
                total += 1
                idx += target_lengths[i]
    
    accuracy = correct / total if total > 0 else 0
    return accuracy


def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0.0
    
    pbar = tqdm(train_loader, desc="Training")
    for images, targets, target_lengths, _ in pbar:
        images = images.to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)
        
        # Forward pass
        logits = model(images)  # (NUM_CLASSES, B, 32)
        
        # Calculate input lengths (sequence length from model)
        input_lengths = torch.full(
            (logits.size(1),), logits.size(0), dtype=torch.long, device=device
        )
        
        # CTC Loss
        loss = criterion(logits, targets, input_lengths, target_lengths)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': loss.item()})
    
    avg_loss = total_loss / len(train_loader)
    return avg_loss


# Create data loaders
batch_size = 64
train_dataset = LPRDataset(DATA_PATH, augment=True, use_hr=False)
train_size = int(0.95 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_data, val_data = torch.utils.data.random_split(
    train_dataset, [train_size, val_size]
)

train_loader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn
)
val_loader = DataLoader(
    val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
)

print(f"✅ Data loaders created")
print(f"   Training samples: {len(train_data)}")
print(f"   Validation samples: {len(val_data)}")
print(f"   Batch size: {batch_size}")

## 5. Training Pipeline

In [ ]:
class CRNN(torch.nn.Module):
    """Convolutional Recurrent Neural Network for License Plate Recognition"""
    
    def __init__(self, num_classes, dropout=0.3):
        super(CRNN, self).__init__()
        
        # CNN Feature Extractor
        self.cnn = torch.nn.Sequential(
            # Layer 1: 1 -> 64 channels
            torch.nn.Conv2d(1, 64, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),  # 32x128 -> 16x64
            
            # Layer 2: 64 -> 128 channels
            torch.nn.Conv2d(64, 128, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),  # 16x64 -> 8x32
            
            # Layer 3: 128 -> 256 channels
            torch.nn.Conv2d(128, 256, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2, 1)),  # 8x32 -> 4x32
            
            # Layer 4: 256 -> 512 channels
            torch.nn.Conv2d(256, 512, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2, 1))  # 4x32 -> 2x32
        )
        
        # Bidirectional LSTM
        self.rnn = torch.nn.LSTM(
            input_size=512 * 2,  # 512 channels × 2 height
            hidden_size=256,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if dropout > 0 else 0
        )
        
        # Classifier head
        self.fc = torch.nn.Sequential(
            torch.nn.Dropout(dropout),
            torch.nn.Linear(512, num_classes)  # 256 * 2 (bi-directional)
        )
    
    def forward(self, x):
        # x: (B, 1, 32, 128)
        
        # CNN feature extraction
        features = self.cnn(x)  # (B, 512, 2, 32)
        
        # Reshape for RNN: (B, L, C) where L is sequence length
        b, c, h, w = features.size()
        features = features.permute(0, 3, 1, 2)  # (B, 32, 512, 2)
        features = features.reshape(b, w, c * h)  # (B, 32, 1024)
        
        # RNN processing
        rnn_out, _ = self.rnn(features)  # (B, 32, 512)
        
        # Classifier
        logits = self.fc(rnn_out)  # (B, 32, NUM_CLASSES)
        logits = logits.permute(2, 0, 1)  # (NUM_CLASSES, B, 32) for CTC
        
        return logits

# Initialize model
model = CRNN(num_classes=NUM_CLASSES, dropout=0.3)
model = model.to(DEVICE)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ CRNN Model initialized")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model device: {DEVICE}")

## 4. Model Architecture: CRNN (CNN + RNN)

In [ ]:
# Character mapping for OCR
CHARS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
char_to_idx = {c: i + 1 for i, c in enumerate(CHARS)}
idx_to_char = {i + 1: c for i, c in enumerate(CHARS)}
NUM_CLASSES = len(CHARS) + 1  # +1 for CTC blank

print(f"✅ Character set: {CHARS}")
print(f"✅ NUM_CLASSES: {NUM_CLASSES}")

class LPRDataset(Dataset):
    """License Plate Recognition Dataset"""
    
    def __init__(self, data_root, augment=True, scenario_filter=None, use_hr=False):
        """
        Args:
            data_root: path to train/ folder
            augment: if True, apply random augmentations
            scenario_filter: filter by scenario (e.g., "Scenario-B")
            use_hr: if True, use HR images; else use LR images
        """
        self.samples = []
        self.data_root = data_root
        self.augment = augment
        self.scenario_filter = scenario_filter
        self.use_hr = use_hr
        self._load_samples()
    
    def _load_samples(self):
        """Load all sample paths"""
        for scenario in os.listdir(self.data_root):
            scenario_path = os.path.join(self.data_root, scenario)
            if not os.path.isdir(scenario_path):
                continue
            
            if self.scenario_filter and scenario != self.scenario_filter:
                continue
            
            for layout in os.listdir(scenario_path):
                layout_path = os.path.join(scenario_path, layout)
                if not os.path.isdir(layout_path):
                    continue
                
                for track in os.listdir(layout_path):
                    track_path = os.path.join(layout_path, track)
                    if not os.path.isdir(track_path):
                        continue
                    
                    ann_path = os.path.join(track_path, 'annotations.json')
                    if os.path.exists(ann_path):
                        self.samples.append({
                            'track_path': track_path,
                            'track_id': track,
                            'scenario': scenario,
                            'layout': layout
                        })
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        track_path = sample['track_path']
        
        # Load annotation
        ann_path = os.path.join(track_path, 'annotations.json')
        with open(ann_path) as f:
            ann = json.load(f)
        
        plate_text = ann.get('plate_text', '')
        
        # Select image (first LR or HR image)
        img_prefix = 'hr-001.png' if self.use_hr else 'lr-001.png'
        img_path = os.path.join(track_path, img_prefix)
        
        # Load and preprocess image
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (128, 32))  # Resize to fixed size
        image = image.astype(np.float32) / 255.0
        
        # Apply augmentation if training
        if self.augment and np.random.rand() > 0.5:
            image = self._augment(image)
        
        # Convert to tensor
        image = torch.from_numpy(image).unsqueeze(0)  # Add channel dimension
        
        # Convert plate text to indices
        label = [char_to_idx.get(c, 0) for c in plate_text if c in char_to_idx]
        label_tensor = torch.tensor(label, dtype=torch.long)
        label_length = len(label)
        
        return image, label_tensor, label_length, plate_text
    
    def _augment(self, image):
        """Apply random augmentations"""
        # Random brightness
        if np.random.rand() > 0.5:
            brightness = np.random.uniform(0.8, 1.2)
            image = np.clip(image * brightness, 0, 1)
        
        # Random contrast
        if np.random.rand() > 0.5:
            contrast = np.random.uniform(0.8, 1.2)
            image = np.clip((image - 0.5) * contrast + 0.5, 0, 1)
        
        # Random noise
        if np.random.rand() > 0.7:
            noise = np.random.normal(0, 0.02, image.shape)
            image = np.clip(image + noise, 0, 1)
        
        return image

# Test dataset loading
train_dataset = LPRDataset(DATA_PATH, augment=False)
print(f"\n✅ Dataset loaded: {len(train_dataset)} samples")

# Display a sample
sample = train_dataset[0]
print(f"Sample image shape: {sample[0].shape}")
print(f"Sample label: {sample[3]} (indices: {sample[1].tolist()})")

## 3. Data Loading & Dataset Class

In [ ]:
# Visualize sample images
def visualize_samples(data_path, num_samples=3):
    """Visualize sample LR and HR images"""
    fig, axes = plt.subplots(num_samples, 2, figsize=(10, 4*num_samples))
    
    sample_count = 0
    for scenario in os.listdir(data_path):
        scenario_path = os.path.join(data_path, scenario)
        if not os.path.isdir(scenario_path):
            continue
        
        for layout in os.listdir(scenario_path):
            layout_path = os.path.join(scenario_path, layout)
            if not os.path.isdir(layout_path):
                continue
            
            for track in os.listdir(layout_path):
                if sample_count >= num_samples:
                    break
                
                track_path = os.path.join(layout_path, track)
                if not os.path.isdir(track_path):
                    continue
                
                lr_img = os.path.join(track_path, 'lr-001.png')
                hr_img = os.path.join(track_path, 'hr-001.png')
                ann_file = os.path.join(track_path, 'annotations.json')
                
                if os.path.exists(lr_img) and os.path.exists(hr_img):
                    lr = cv2.imread(lr_img, cv2.IMREAD_GRAYSCALE)
                    hr = cv2.imread(hr_img, cv2.IMREAD_GRAYSCALE)
                    
                    axes[sample_count, 0].imshow(lr, cmap='gray')
                    axes[sample_count, 0].set_title(f"LR: {lr.shape} (Scenario: {scenario})")
                    axes[sample_count, 0].axis('off')
                    
                    axes[sample_count, 1].imshow(hr, cmap='gray')
                    axes[sample_count, 1].set_title(f"HR: {hr.shape}")
                    axes[sample_count, 1].axis('off')
                    
                    if os.path.exists(ann_file):
                        with open(ann_file) as f:
                            ann = json.load(f)
                            plate_text = ann.get('plate_text', 'N/A')
                            layout = ann.get('plate_layout', 'N/A')
                            fig.text(0.5, 0.02 + sample_count*0.15, 
                                    f"Track: {track} | Plate: {plate_text} | Layout: {layout}",
                                    ha='center', fontsize=10)
                    
                    sample_count += 1
            
            if sample_count >= num_samples:
                break
    
    plt.tight_layout()
    plt.savefig('/content/output/sample_images.png', dpi=100, bbox_inches='tight')
    plt.show()
    print("✅ Sample images saved to /content/output/sample_images.png")

visualize_samples(DATA_PATH, num_samples=3)

In [ ]:
# Data exploration function
def explore_dataset(data_path):
    """Explore dataset structure and statistics"""
    stats = {
        'total_tracks': 0,
        'scenarios': {},
        'plate_layouts': {'Brazilian': 0, 'Mercosur': 0},
        'plate_texts': [],
        'image_sizes': {'lr': [], 'hr': []}
    }
    
    for scenario in os.listdir(data_path):
        scenario_path = os.path.join(data_path, scenario)
        if not os.path.isdir(scenario_path):
            continue
        
        stats['scenarios'][scenario] = 0
        
        for layout in os.listdir(scenario_path):
            layout_path = os.path.join(scenario_path, layout)
            if layout in ['Brazilian', 'Mercosur']:
                stats['plate_layouts'][layout] += 1
            
            if not os.path.isdir(layout_path):
                continue
            
            for track in os.listdir(layout_path):
                track_path = os.path.join(layout_path, track)
                if not os.path.isdir(track_path):
                    continue
                
                ann_file = os.path.join(track_path, 'annotations.json')
                if os.path.exists(ann_file):
                    with open(ann_file) as f:
                        ann = json.load(f)
                        stats['plate_texts'].append(ann.get('plate_text', ''))
                
                # Get image sizes
                for img_file in os.listdir(track_path):
                    if img_file.endswith('.png'):
                        img_path = os.path.join(track_path, img_file)
                        try:
                            img = Image.open(img_path)
                            if img_file.startswith('lr-'):
                                stats['image_sizes']['lr'].append(img.size)
                            elif img_file.startswith('hr-'):
                                stats['image_sizes']['hr'].append(img.size)
                        except:
                            pass
                
                stats['total_tracks'] += 1
                stats['scenarios'][scenario] += 1
    
    return stats

print("🔍 Exploring dataset...")
dataset_stats = explore_dataset(DATA_PATH)

print(f"\n✅ Dataset Statistics:")
print(f"Total tracks: {dataset_stats['total_tracks']}")
print(f"Scenarios: {dataset_stats['scenarios']}")
print(f"Plate layouts: {dataset_stats['plate_layouts']}")
print(f"Unique plate texts: {len(set(dataset_stats['plate_texts']))}")
if dataset_stats['image_sizes']['lr']:
    print(f"LR image size: {dataset_stats['image_sizes']['lr'][0]} (samples: {len(dataset_stats['image_sizes']['lr'])})")
if dataset_stats['image_sizes']['hr']:
    print(f"HR image size: {dataset_stats['image_sizes']['hr'][0]} (samples: {len(dataset_stats['image_sizes']['hr'])})")

## 2. Data Exploration & Analysis

In [ ]:
# Configure paths
DRIVE_PATH = '/content/drive/MyDrive/LPR_Project'
DATA_PATH = '/content/drive/MyDrive/LPR_Project/data/raw/wYe7pBJ7-train/train'

# Create working directories
os.makedirs('/content/output', exist_ok=True)
os.makedirs('/content/checkpoints', exist_ok=True)

print(f"Drive path: {DRIVE_PATH}")
print(f"Data path: {DATA_PATH}")
print(f"\nDirectory structure:")
print(f"Data exists: {os.path.exists(DATA_PATH)}")
if os.path.exists(DATA_PATH):
    print(f"Contents: {os.listdir(DATA_PATH)}")

In [ ]:
# Import libraries
import os
import json
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check GPU availability
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using device: {DEVICE}")
if DEVICE == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Install required dependencies
import subprocess
import sys

packages = [
    'torch',
    'torchvision',
    'opencv-python',
    'pillow',
    'numpy',
    'tqdm',
    'scikit-image',
    'paddleocr',
    'pyyaml',
]

print("Installing packages...")
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
print("✅ All dependencies installed")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully")

## 1. Setup and Data Loading

# ICPR 2026: Low-Resolution License Plate Recognition
## Complete End-to-End Solution

This notebook provides a complete workflow for the ICPR 2026 competition using:
- **Super-Resolution**: Upscaling low-res license plate images
- **OCR**: Character recognition on enhanced images
- **Aggregation**: Combining predictions across multiple frames

**Target**: Achieve >60% accuracy on blind test set